# Elastic Stackの構築手順：収容（AWS）

----

Elasticsearchのインストール先となる Amazon&nbsp;EC2インスタンスを確保します。  

**事前に [01_01_Outline.ipynb#設定ファイルの出力](01_01_Outline.ipynb#設定ファイルの出力) を実行しておく必要があります。**


In [1]:
import os
assert os.path.exists('group_vars/all')


**上記がエラーになる場合は、[01_01_Outline.ipynb#設定ファイルの出力](01_01_Outline.ipynb#設定ファイルの出力)を実行してください。**

## 設定
インスタンスの生成に必要な情報を設定します。

マシンイメージを設定します。

In [2]:
image_id = 'ami-eec1c380'
#CentOS 7 (x86_64) - with Updates HVM
#アジアパシフィック東京リージョン

インスタンスタイプを設定します。  
[Amazon EC2 インスタンス](https://aws.amazon.com/jp/ec2/instance-types/)のページから適切なタイプを選択してください。

例えば64GBのメモリを利用するのであれば **m4.4xlarge** が選択候補となります。  
利用したいサイズに応じてタイプを選んで下さい。

**<font color="red">デモ環境の場合は変更しないでください。</font>**

In [3]:
instance_type = 'm4.4xlarge'

In [4]:
location = 'Asia Pacific (Tokyo)'

In [5]:
instance_count = 1

データ蓄積ディスクとなる[EBS](http://docs.aws.amazon.com/ja_jp/AWSEC2/latest/UserGuide/AmazonEBS.html)の[ブロックデバイスマッピング](http://docs.aws.amazon.com/ja_jp/AWSEC2/latest/UserGuide/block-device-mapping-concepts.html)設定を行います。  
`VolumeSize`、`VolumeType`を設定します。  
`DeleteOnTermination`にtrueを設定すると、インスタンス破棄時にEBSが削除されます。  


In [6]:
block_device_mappings = {
    "DeviceName": "/dev/sda1",
    "Ebs": {
      "VolumeSize": 1024,
      "VolumeType": "gp2",
      "DeleteOnTermination": True,
    }
}

使用するキーペア名を設定します。

In [7]:
import os
%run {os.environ['HOST_HOME']}/env.py
keypair_name

このキーペアには以下の公開鍵が設定されています。

In [8]:
!cat ~/.ssh/ansible_id_rsa.pub

**<font color="red">以下3つの設定はデモ環境の場合は変更しないでください。</font>**

In [9]:
security_group_id = 'sg-xxxxxxxx'

In [10]:
vpc_id = 'vpc-xxxxxxxx'

In [11]:
subnet_id = 'subnet-xxxxxxxx'

## EC2インスタンスの設定と生成 
サーバ構成に応じたEC2インスタンスの設定を定義し、インスタンスを生成します。

大まかな手順は次の通りです。  
1. AWSへのアクセスの確認
2. インスタンスの生成
3. セキュリティグループへの登録
4. タグの設定


### インスタンスの生成

設定した定義を用いてインスタンスを生成します。このデモでは、AWSのスポットインスタンスを利用します。

まず、AWSに伝えるスポットインスタンスの定義ファイルを生成します。

In [12]:
import json
import tempfile
import os

work_dir = tempfile.mkdtemp()
spec = {'ImageId': image_id, 'KeyName': keypair_name, 'SecurityGroupIds': [security_group_id],
        'BlockDeviceMappings': [block_device_mappings], 'SubnetId': subnet_id,
        'InstanceType': instance_type}
with open(os.path.join(work_dir, 'spec.json'), 'w') as f:
    json.dump(spec, f)
!cat {work_dir}/spec.json

定義ファイルに基づいて、AWSにスポットインスタンス要求を発行します。

In [13]:
req_instances_stdout = !aws ec2 request-spot-instances --spot-price "0.5" \
                            --instance-count {instance_count} --type one-time \
                            --launch-specification file://{work_dir}/spec.json
try:
    json.loads('\n'.join(req_instances_stdout))
except:
    print('\n'.join(req_instances_stdout))

req_ids = [r['SpotInstanceRequestId'] for r in json.loads('\n'.join(req_instances_stdout))['SpotInstanceRequests']]
req_ids

インスタンス要求が受理されるまで待機します。

In [14]:
import time

retries = 10
while retries > 0:
    desc_requests_stdout = !aws ec2 describe-spot-instance-requests --spot-instance-request-ids {' '.join(req_ids)}
    try:
        json.loads('\n'.join(desc_requests_stdout))
    except:
        print('\n'.join(desc_requests_stdout))
    
    reqs = json.loads('\n'.join(desc_requests_stdout))['SpotInstanceRequests']
    fulfilled = [req for req in reqs if req['Status']['Code'] == 'fulfilled']
    if len(fulfilled) == len(req_ids):
        break
    print('Not accepted : {}/{}'.format(len(fulfilled), len(req_ids)))
    time.sleep(5)
    print('------')
    retries -= 1

# Statusを確認
for req in reqs:
    assert req['Status']['Code'] == 'fulfilled'
id_list = [r['InstanceId'] for r in reqs]
id_list

生成したインスタンスIDを確認します。

In [15]:
id_list_text = ' '.join(id_list)
id_list

インスタンスの状態が `running` になるまで待ちます。

In [16]:
from functools import reduce
import time

retries = 10
while retries > 0:
    # インスタンス情報を取得
    desc_instances_stdout = !aws ec2 describe-instances --instance-ids $id_list_text
    try:
        json.loads('\n'.join(desc_instances_stdout))
    except:
        print('\n'.join(desc_instances_stdout))
    
    # JSON化し、インスタンスのリストとして取り出す
    desc_instances = json.loads('\n'.join(desc_instances_stdout))
    target_instances = reduce(lambda x, y: x + y, map(lambda r: r['Instances'], desc_instances['Reservations']))
    # IDと状態を出力
    status_text = '\n'.join(map(lambda i : '{0} : {1}'.format(i['InstanceId'], i['State']['Name']), target_instances))
    print(status_text)
    # runningになっていないものがまだあれば、スリープして最初に戻る。全てrunningなら終了。
    not_running_num = len(list(filter(lambda i : i['State']['Name'] != 'running', target_instances)))
    if not_running_num == 0:
        break
    print('Not running : {0}'.format(not_running_num))
    time.sleep(5)
    print('------')
    retries -= 1

for i in target_instances:
    assert i['State']['Name'] == 'running'
host_list = [i['PrivateIpAddress'] for i in target_instances]
host_list

pingが通ることを確認します。

Instanceがrunningになった後、**<font color="red">pingが疎通するようになるまでしばらく(数分程度)</font>**かかる場合もあります。pingの実行が成功する(packet lossが0%)まで以下のセルの実行を試すようにしてください。

In [18]:
for host in host_list:
    !ping -c 4 {host}

全てのインスタンスに対しpingが通ればEC2インスタンス生成は完了です。

## Inventory作成用のインスタンスリストを作成
ホストのパブリックDNS一覧をリスト化します  

次のブックの[Inventory作成](01_03_Set_Inventory.ipynb)で使用します。  
出現した文字列をクリップボードにコピーしてください。

In [19]:
host_list = ['\'host{0}\':\'{1}\''.format(str(i+1), x) for i,x in enumerate(host_list)] 
print('host_list = {{{0}}}'.format(','.join(host_list)))